# Spark integration with Azure Cognitive Services

At Spark + AI Summit 2019, Microsoft announced a new set of models in the SparkML ecosystem that make it easy to leverage the Azure Cognitive Services at terabyte scales. With only a few lines of code, developers can embed cognitive services within your existing distributed machine learning pipelines in Spark ML. Additionally, these contributions allow Spark users to chain or Pipeline services together with deep networks, gradient boosted trees, and any SparkML model and apply these hybrid models in elastic and serverless distributed systems.

From image recognition to object detection using speech recognition, translation, and text-to-speech, Azure Cognitive Services makes it easy for developers to add intelligent capabilities to their applications in any scenario. This notebook demostrates the integration of PySpark (using Azure Databricks) with Azure Cognitive Service [Text Analytics](https://azure.microsoft.com/en-us/services/cognitive-services/text-analytics/) to extract valuable information from text data. 

## Prerequisites

- Spark 2.4 environment
  - You can use Azure Databricks for an integrated Spark environment
- Install required libraries in Spark
  - [MMLSpark](https://mmlspark.blob.core.windows.net/website/index.html#install)
  - [azure-cognitiveservices-language-textanalytics](https://pypi.org/project/azure-cognitiveservices-language-textanalytics/)
- Create [Azure Cognitive Services multi-service resource](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Clinux)
- Import [Customers sample dataset](https://github.com/kawo123/azure-databricks/blob/master/data/customers.csv) into Spark environment

## References

- [Spark and Azure Cognitive Services blog](https://azure.microsoft.com/en-us/blog/dear-spark-developers-welcome-to-azure-cognitive-services/)

In [2]:
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials
from mmlspark.cognitive import TextSentiment
from pyspark.sql.functions import col

In [3]:
# Obtain Azure Text Analytics endpoint and key. Replace <<TODO>> below with your endpoint and key
textanalytics_endpoint = '<<TODO>>' # TODO
textanalytics_key = '<<TODO>>' # TODO

# Initialize Azure Text Analytics client
client = TextAnalyticsClient(textanalytics_endpoint, CognitiveServicesCredentials(textanalytics_key))

In [4]:
# Create sample text documents for analysis
docs = [
  { 'id': '1', 'language': 'en', 'text': 'This is awesome!' },
  { 'id': '2', 'language': 'en', 'text': 'This was a waste of my time. The speaker put me to sleep.' },
  { 'id': '3', 'language': 'en', 'text': None },
  { 'id': '4', 'language': 'en', 'text': 'Hello World' }
]

# Submit text documents for sentiment analysis
resp = client.sentiment(documents=docs)

# Print sentiment analysis results
for document in resp.documents:
    print("Document Id: ", document.id, ", Sentiment Score: ", "{:.2f}".format(document.score))

Document Id: 1 , Sentiment Score: 1.00
Document Id: 2 , Sentiment Score: 0.11
Document Id: 4 , Sentiment Score: 0.76

You should observe output similar to below

```
Document Id:  1 , Sentiment Score:  1.00
Document Id:  2 , Sentiment Score:  0.11
Document Id:  4 , Sentiment Score:  0.76
```

In [6]:
# Read customers csv
df_customers = spark\
  .read\
  .option('header', True)\
  .option('inferSchema', True)\
  .csv('/FileStore/tables/customers.csv')

df_customers.show(2)

+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+
Customer grouping|Product ID| State|Customer category|Product market price|total market price|Notes| store comments| Customer Address|Gender|Discount| Date|Quantity|Discount_discrete|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+
 4.0| v15| Oklahoma| Automotive| 45000| 2899183| null|This is the best ...|295 WILLIS AVENUE...| 1| 56|42734| 91| (50, 60]|
 5.0| v11|South Carolina| Entertainment| 238670| 1023162| null|This is the only ...|4454 IDEA CENTER ...| 1| 50|43090| 26| (40, 50]|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+
only showing top 2 rows

You should see a table with the following columns: `Customer grouping`, `Product ID`, `State`, `Customer category`, `Product market price`, `total market price`, `Notes`, `store comments`, `Customer Address`, `Gender`, `Discount`, `Date`, `Quantity`, `Discount_discrete`

In [8]:
# Define Sentiment Analysis pipeline
pipe_text_sentiment = (TextSentiment()
  .setSubscriptionKey(textanalytics_key)
  .setLocation('eastus')
  .setLanguage('en')
  .setTextCol('store comments')
  .setOutputCol("StoreCommentSentimentObj")
  .setErrorCol("Errors")
  .setConcurrency(10)
)

# Process df_customers with the Sentiment Analysis pipeline 
df_customers_sentiment = pipe_text_sentiment.transform(df_customers)

df_customers_sentiment.show(2)

+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+------+------------------------+
Customer grouping|Product ID| State|Customer category|Product market price|total market price|Notes| store comments| Customer Address|Gender|Discount| Date|Quantity|Discount_discrete|Errors|StoreCommentSentimentObj|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+------+------------------------+
 4.0| v15| Oklahoma| Automotive| 45000| 2899183| null|This is the best ...|295 WILLIS AVENUE...| 1| 56|42734| 91| (50, 60]| null| [[0.8554108,]]|
 5.0| v11|South Carolina| Entertainment| 238670| 1023162| null|This is the only ...|4454 IDEA CENTER ...| 1| 50|43090| 26| (40, 50]| null| [[0.25904304,]]|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+------+------------------------+
only showing top 2 rows

After the customer dataframe is processed by the sentiment analysis pipeline, you should see 2 additional columns in the table: `Errors` and `StoreCommentSentimentObj`. The `Errors` column contains any error message that the text analytics pipeline encounter. The `StoreCommentSentimentObj` column contains the an array of sentiment objects returned by the Text Analytics service. The sentiment object includes sentiment score and any error messages that the Text Analytics engine encounters.

In [10]:
# Extract sentiment score from store comment sentiment complex objects
df_customers_sentiment_numeric = (df_customers_sentiment
  .select('*', col('StoreCommentSentimentObj').getItem(0).getItem('score').alias('StoreCommentSentimentScore'))
  .drop('Errors', 'StoreCommentSentimentObj')
)

df_customers_sentiment_numeric.show(2)

+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+--------------------------+
Customer grouping|Product ID| State|Customer category|Product market price|total market price|Notes| store comments| Customer Address|Gender|Discount| Date|Quantity|Discount_discrete|StoreCommentSentimentScore|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+--------------------------+
 4.0| v15| Oklahoma| Automotive| 45000| 2899183| null|This is the best ...|295 WILLIS AVENUE...| 1| 56|42734| 91| (50, 60]| 0.8554108|
 5.0| v11|South Carolina| Entertainment| 238670| 1023162| null|This is the only ...|4454 IDEA CENTER ...| 1| 50|43090| 26| (40, 50]| 0.25904304|
+-----------------+----------+--------------+-----------------+--------------------+------------------+-----+--------------------+--------------------+------+--------+-----+--------+-----------------+--------------------------+
only showing top 2 rows

You should now see a new column `StoreCommentSentimentScore` which contains the numeric sentiment scores of store comments